In [1]:
#import necessary liabries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style ="darkgrid")
%matplotlib inline
import os

In [2]:
#define a Function that will read multiple csv docs and concatenate the dfs into one
def read_to_dataframe(path, csv_list):
    """
    source: Ryan
    input: path and csv_list
    """
    df_list=[]
    for i in csv_list:
        df_list.append(pd.read_csv(path.format(i)))
    return pd.concat(df_list,sort=True)

In [3]:
#define a Function that will remove Unused leagues
def remove_unused_leagues(df,new_leagues):
    """
    remove unused rows from a dataframe
    input = df and new_leagues
    """
    return df[df['league'].isin(new_leagues)]
new_leagues = ['Barclays Premier League','UEFA Champions League',
                   'Major League Soccer','Italy Serie A','Spanish Primera Division',
                   'German Bundesliga','French Ligue 1']

# FiveThirtyEight Dataset

In [4]:
# read the Five thirty data with four folders
spi_data_intl_df = pd.read_csv("../Data/soccer-spi/spi_global_rankings_intl.csv")
spi_data_df = pd.read_csv("../Data/soccer-spi/spi_global_rankings.csv")
spi_matches_latest_df = pd.read_csv("../Data/soccer-spi/spi_matches_latest.csv")
spi_matches_df =pd.read_csv("../Data/soccer-spi/spi_matches.csv")


In [5]:
#check the first five rows
spi_data_intl_df.head()

,rank,name,confed,off,def,spi
0,1,Brazil,CONMEBOL,2.92,0.26,92.38
1,2,Spain,UEFA,2.97,0.41,90.41
2,3,Argentina,CONMEBOL,2.88,0.38,90.17
3,4,Germany,UEFA,3.27,0.58,90.03
4,5,France,UEFA,2.74,0.40,88.69


Since the spi data Intl contains confederation soccer, I will not be using it to build my model. My focus is more on the leagues.

In [6]:
#check the first five rows
spi_data_df.head(5)

,rank,prev_rank,name,league,off,def,spi
0,1,1,Bayern Munich,German Bundesliga,3.64,0.51,93.72
1,2,2,Manchester City,Barclays Premier League,2.97,0.24,93.57
2,3,3,Liverpool,Barclays Premier League,3.06,0.46,91.04
3,4,4,Chelsea,Barclays Premier League,2.47,0.24,90.32
4,5,5,Ajax,Dutch Eredivisie,3.05,0.59,89.12


In [7]:
#check the shape
spi_data_df.shape

(645, 7)

In [8]:
#value count the leagues
spi_data_df.league.value_counts()

United Soccer League                        31
Major League Soccer                         27
Argentina Primera Division                  26
English League One                          24
English League Two                          24
English League Championship                 24
Spanish Segunda Division                    22
Barclays Premier League                     20
French Ligue 1                              20
Italy Serie A                               20
Italy Serie B                               20
Turkish Turkcell Super Lig                  20
Japanese J League                           20
French Ligue 2                              20
Spanish Primera Division                    20
Brasileiro Série A                          20
Mexican Primera Division Torneo Apertura    18
Dutch Eredivisie                            18
UEFA Europa Conference League               18
German 2. Bundesliga                        18
German Bundesliga                           18
Portuguese Li

In [9]:
#check the data information to see if 
#there are null values and the data types
spi_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rank       645 non-null    int64  
 1   prev_rank  645 non-null    int64  
 2   name       645 non-null    object 
 3   league     645 non-null    object 
 4   off        645 non-null    float64
 5   def        645 non-null    float64
 6   spi        645 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 35.4+ KB


In [10]:
#checking the sum of null values
spi_data_df.isna().sum()

rank         0
prev_rank    0
name         0
league       0
off          0
def          0
spi          0
dtype: int64

In [11]:
#remove unused league
spi_data_df = remove_unused_leagues(spi_data_df,new_leagues)
spi_data_df.head()

,rank,prev_rank,name,league,off,def,spi
0,1,1,Bayern Munich,German Bundesliga,3.64,0.51,93.72
1,2,2,Manchester City,Barclays Premier League,2.97,0.24,93.57
2,3,3,Liverpool,Barclays Premier League,3.06,0.46,91.04
3,4,4,Chelsea,Barclays Premier League,2.47,0.24,90.32
5,6,6,Real Madrid,Spanish Primera Division,2.57,0.62,84.21


In [12]:
spi_data_df.shape

(128, 7)

In [13]:
#checking if one of the index in league to make sure no data was lost
spi_data_df[spi_data_df['league'] == 'UEFA Champions League']

,rank,prev_rank,name,league,off,def,spi
71,72,71,Shakhtar Donetsk,UEFA Champions League,1.99,1.13,64.64
112,113,113,Dynamo Kiev,UEFA Champions League,1.62,1.13,57.06
214,215,214,FC Sheriff Tiraspol,UEFA Champions League,1.42,1.42,45.86


In [14]:
spi_data_df.league.value_counts()

Major League Soccer         27
Barclays Premier League     20
French Ligue 1              20
Italy Serie A               20
Spanish Primera Division    20
German Bundesliga           18
UEFA Champions League        3
Name: league, dtype: int64

In [15]:
#reset the index
spi_data_df.reset_index(drop=True)

,rank,prev_rank,name,league,off,def,spi
0,1,1,Bayern Munich,German Bundesliga,3.64,0.51,93.72
1,2,2,Manchester City,Barclays Premier League,2.97,0.24,93.57
2,3,3,Liverpool,Barclays Premier League,3.06,0.46,91.04
3,4,4,Chelsea,Barclays Premier League,2.47,0.24,90.32
4,6,6,Real Madrid,Spanish Primera Division,2.57,0.62,84.21
...,...,...,...,...,...,...,...
123,404,407,Inter Miami CF,Major League Soccer,1.10,1.68,32.92
124,418,419,Toronto FC,Major League Soccer,1.19,1.84,32.35
125,434,435,Austin FC,Major League Soccer,1.13,1.81,31.39
126,436,437,Houston Dynamo,Major League Soccer,1.07,1.73,31.31


In [16]:
#check the first five rows
spi_matches_latest_df.head(5)

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2019,2019-03-01,1979,Chinese Super League,Shandong Luneng,Guizhou Renhe,48.22,37.83,0.5755,0.1740,...,45.9,22.1,1.0,0.0,1.39,0.26,2.05,0.54,1.05,0.00
1,2019,2019-03-01,1979,Chinese Super League,Guangzhou Evergrande,Tianjin Quanujian,65.59,39.99,0.7832,0.0673,...,77.1,28.8,3.0,0.0,0.49,0.45,1.05,0.75,3.15,0.00
2,2019,2019-03-01,1979,Chinese Super League,Shanghai Greenland,Shanghai SIPG,39.81,60.08,0.2387,0.5203,...,25.6,63.4,0.0,4.0,0.57,2.76,0.80,1.50,0.00,3.26
3,2019,2019-03-01,1979,Chinese Super League,Wuhan Zall,Beijing Guoan,32.25,54.82,0.2276,0.5226,...,35.8,58.9,0.0,1.0,1.12,0.97,1.51,0.94,0.00,1.05
4,2019,2019-03-01,1979,Chinese Super League,Chongqing Lifan,Guangzhou RF,38.24,40.45,0.4403,0.2932,...,26.2,21.3,2.0,2.0,2.77,3.17,1.05,2.08,2.10,2.10


In [17]:
#convert the date to python datetime 
spi_matches_latest_df['date']=pd.to_datetime(spi_matches_latest_df['date'])
spi_matches_latest_df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2019,2019-03-01,1979,Chinese Super League,Shandong Luneng,Guizhou Renhe,48.22,37.83,0.5755,0.1740,...,45.9,22.1,1.0,0.0,1.39,0.26,2.05,0.54,1.05,0.00
1,2019,2019-03-01,1979,Chinese Super League,Guangzhou Evergrande,Tianjin Quanujian,65.59,39.99,0.7832,0.0673,...,77.1,28.8,3.0,0.0,0.49,0.45,1.05,0.75,3.15,0.00
2,2019,2019-03-01,1979,Chinese Super League,Shanghai Greenland,Shanghai SIPG,39.81,60.08,0.2387,0.5203,...,25.6,63.4,0.0,4.0,0.57,2.76,0.80,1.50,0.00,3.26
3,2019,2019-03-01,1979,Chinese Super League,Wuhan Zall,Beijing Guoan,32.25,54.82,0.2276,0.5226,...,35.8,58.9,0.0,1.0,1.12,0.97,1.51,0.94,0.00,1.05
4,2019,2019-03-01,1979,Chinese Super League,Chongqing Lifan,Guangzhou RF,38.24,40.45,0.4403,0.2932,...,26.2,21.3,2.0,2.0,2.77,3.17,1.05,2.08,2.10,2.10


In [18]:
spi_matches_latest_df.shape

(11447, 23)

In [19]:
spi_matches_latest_df.league.value_counts()

English League Championship                 552
English League One                          552
English League Two                          552
United Soccer League                        510
Major League Soccer                         465
Spanish Segunda Division                    462
Barclays Premier League                     380
Brasileiro Série A                          380
Italy Serie A                               380
Italy Serie B                               380
French Ligue 1                              380
Spanish Primera Division                    380
French Ligue 2                              380
Japanese J League                           380
Turkish Turkcell Super Lig                  380
Argentina Primera Division                  325
German Bundesliga                           306
German 2. Bundesliga                        306
Dutch Eredivisie                            306
Portuguese Liga                             306
Belgian Jupiler League                  

In [20]:
#check the data information to see if 
#there are null values and the data types
spi_matches_latest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11447 entries, 0 to 11446
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   season       11447 non-null  int64         
 1   date         11447 non-null  datetime64[ns]
 2   league_id    11447 non-null  int64         
 3   league       11447 non-null  object        
 4   team1        11447 non-null  object        
 5   team2        11447 non-null  object        
 6   spi1         11447 non-null  float64       
 7   spi2         11447 non-null  float64       
 8   prob1        11447 non-null  float64       
 9   prob2        11447 non-null  float64       
 10  probtie      11447 non-null  float64       
 11  proj_score1  11447 non-null  float64       
 12  proj_score2  11447 non-null  float64       
 13  importance1  6179 non-null   float64       
 14  importance2  6179 non-null   float64       
 15  score1       6290 non-null   float64       
 16  scor

In [21]:
spi_matches_latest_df.isna().sum()

season            0
date              0
league_id         0
league            0
team1             0
team2             0
spi1              0
spi2              0
prob1             0
prob2             0
probtie           0
proj_score1       0
proj_score2       0
importance1    5268
importance2    5268
score1         5157
score2         5157
xg1            7758
xg2            7758
nsxg1          7758
nsxg2          7758
adj_score1     7758
adj_score2     7758
dtype: int64

In [22]:
#remove unused league
spi_matches_latest_df =remove_unused_leagues(spi_matches_latest_df,new_leagues)
spi_matches_latest_df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
566,2021,2021-04-16,1951,Major League Soccer,Houston Dynamo,San Jose Earthquakes,36.30,35.71,0.4861,0.2884,...,20.7,21.8,2.0,1.0,1.27,1.06,1.38,0.54,2.10,1.05
568,2021,2021-04-16,1951,Major League Soccer,Seattle Sounders FC,Minnesota United FC,48.14,41.11,0.5544,0.2208,...,13.8,19.6,4.0,0.0,2.63,0.90,1.70,1.26,3.73,0.00
579,2021,2021-04-17,1951,Major League Soccer,Montreal Impact,Toronto FC,30.18,39.67,0.3701,0.3891,...,18.3,21.2,4.0,2.0,1.01,2.25,0.66,1.08,4.18,2.10
580,2021,2021-04-17,1951,Major League Soccer,Orlando City SC,Atlanta United FC,40.56,37.41,0.5021,0.2466,...,21.2,20.5,0.0,0.0,1.53,1.11,2.15,1.34,0.00,0.00
581,2021,2021-04-17,1951,Major League Soccer,Los Angeles FC,Austin FC,50.97,27.43,0.7083,0.1130,...,14.2,15.5,2.0,0.0,2.40,0.58,2.36,1.23,1.58,0.00


In [23]:
#check the shape of the data
spi_matches_latest_df.shape

(2387, 23)

In [24]:
#checking the UEFA league to see if am missing the data
spi_matches_latest_df[spi_matches_latest_df['league'] == 'UEFA Champions League'].head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
3366,2021,2021-09-14,1818,UEFA Champions League,Sevilla FC,FC Salzburg,80.64,78.05,0.4886,0.2709,...,79.3,78.9,1.0,1.0,1.47,2.79,1.16,0.99,1.05,1.05
3367,2021,2021-09-14,1818,UEFA Champions League,Young Boys,Manchester United,65.12,85.60,0.1980,0.5777,...,53.0,55.6,2.0,1.0,1.46,0.64,1.29,0.59,1.68,1.05
3377,2021,2021-09-14,1818,UEFA Champions League,Villarreal,Atalanta,75.69,78.48,0.4013,0.3385,...,76.0,76.3,2.0,2.0,2.66,1.03,1.20,1.08,2.10,2.10
3378,2021,2021-09-14,1818,UEFA Champions League,Barcelona,Bayern Munich,87.62,91.37,0.3796,0.4120,...,54.1,43.6,0.0,3.0,0.23,2.56,0.41,2.34,0.00,2.77
3380,2021,2021-09-14,1818,UEFA Champions League,Dynamo Kiev,Benfica,56.83,78.64,0.1781,0.5825,...,21.5,54.1,0.0,0.0,0.59,1.22,0.36,1.11,0.00,0.00


In [25]:
spi_matches_latest_df.league.value_counts()

Major League Soccer         465
Barclays Premier League     380
Spanish Primera Division    380
French Ligue 1              380
Italy Serie A               380
German Bundesliga           306
UEFA Champions League        96
Name: league, dtype: int64

In [26]:
#check the first five rows
spi_matches_df.head(5)

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#convert date to dataetime python
spi_matches_df['date']=pd.to_datetime(spi_matches_df['date'])
spi_matches_df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
spi_matches_df.shape

(54032, 23)

In [29]:
spi_matches_df.league.value_counts()

English League Championship                 2780
Spanish Segunda Division                    2333
Barclays Premier League                     2280
French Ligue 1                              2280
Italy Serie A                               2280
Spanish Primera Division                    2280
English League Two                          2111
English League One                          2071
Major League Soccer                         2009
United Soccer League                        2007
Italy Serie B                               1982
Brasileiro Série A                          1900
French Ligue 2                              1900
German Bundesliga                           1836
Turkish Turkcell Super Lig                  1718
Portuguese Liga                             1530
German 2. Bundesliga                        1530
Dutch Eredivisie                            1530
Argentina Primera Division                  1304
Norwegian Tippeligaen                       1200
Swedish Allsvenskan 

In [30]:
#check the data information to see if 
#there are null values and the data types
spi_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54032 entries, 0 to 54031
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   season       54032 non-null  int64         
 1   date         54032 non-null  datetime64[ns]
 2   league_id    54032 non-null  int64         
 3   league       54032 non-null  object        
 4   team1        54032 non-null  object        
 5   team2        54032 non-null  object        
 6   spi1         54032 non-null  float64       
 7   spi2         54032 non-null  float64       
 8   prob1        54032 non-null  float64       
 9   prob2        54032 non-null  float64       
 10  probtie      54032 non-null  float64       
 11  proj_score1  54032 non-null  float64       
 12  proj_score2  54032 non-null  float64       
 13  importance1  43104 non-null  float64       
 14  importance2  43104 non-null  float64       
 15  score1       48302 non-null  float64       
 16  scor

In [31]:
spi_matches_df.isna().sum()

season             0
date               0
league_id          0
league             0
team1              0
team2              0
spi1               0
spi2               0
prob1              0
prob2              0
probtie            0
proj_score1        0
proj_score2        0
importance1    10928
importance2    10928
score1          5730
score2          5730
xg1            28096
xg2            28096
nsxg1          28096
nsxg2          28096
adj_score1     28096
adj_score2     28096
dtype: int64

In [32]:
#remove unused league
spi_matches_df = remove_unused_leagues(spi_matches_df,new_leagues)
spi_matches_df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
10,2016,2016-08-12,1843,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,...,32.4,67.7,0.0,1.0,0.97,0.63,0.43,0.45,0.00,1.05
11,2016,2016-08-12,1843,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,...,53.7,22.9,2.0,2.0,2.45,0.77,1.75,0.42,2.10,2.10
12,2016,2016-08-13,2411,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,...,38.1,22.2,2.0,1.0,0.85,2.77,0.17,1.25,2.10,1.05
13,2016,2016-08-13,2411,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,...,31.9,48.0,1.0,1.0,0.73,1.11,0.88,1.81,1.05,1.05
14,2016,2016-08-13,2411,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,...,43.6,34.6,0.0,1.0,1.11,0.68,0.84,1.60,0.00,1.05


In [33]:
spi_matches_df.shape

(13680, 23)

In [34]:
spi_matches_df[spi_matches_df['league'] == 'UEFA Champions League'].head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
174,2016,2016-09-13,1818,UEFA Champions League,Barcelona,Celtic,95.90,62.22,0.8787,0.0398,...,NaN,NaN,7.0,0.0,3.82,0.78,3.88,0.05,6.77,0.00
175,2016,2016-09-13,1818,UEFA Champions League,Bayern Munich,Rostov,94.80,59.97,0.9179,0.0155,...,NaN,NaN,5.0,0.0,3.31,0.21,2.25,0.86,4.74,0.00
176,2016,2016-09-13,1818,UEFA Champions League,Benfica,Besiktas,74.49,59.93,0.6938,0.1159,...,NaN,NaN,1.0,1.0,1.45,0.72,1.17,1.02,1.05,1.05
177,2016,2016-09-13,1818,UEFA Champions League,Basel,Ludogorets,70.61,50.99,0.3468,0.3606,...,NaN,NaN,1.0,1.0,0.81,0.49,1.24,0.38,1.05,1.05
178,2016,2016-09-13,1818,UEFA Champions League,Dynamo Kiev,Napoli,80.70,76.79,0.3504,0.3697,...,NaN,NaN,1.0,2.0,0.46,1.43,0.48,0.64,1.05,2.10


In [35]:
spi_matches_df.league.value_counts()

Barclays Premier League     2280
Spanish Primera Division    2280
French Ligue 1              2280
Italy Serie A               2280
Major League Soccer         2009
German Bundesliga           1836
UEFA Champions League        715
Name: league, dtype: int64

In [36]:
#merge the spi matches to the latest matches
matches_df = pd.merge(spi_matches_df,spi_matches_latest_df, how='outer')
matches_df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-08-12,1843,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,...,32.4,67.7,0.0,1.0,0.97,0.63,0.43,0.45,0.00,1.05
1,2016,2016-08-12,1843,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,...,53.7,22.9,2.0,2.0,2.45,0.77,1.75,0.42,2.10,2.10
2,2016,2016-08-13,2411,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,...,38.1,22.2,2.0,1.0,0.85,2.77,0.17,1.25,2.10,1.05
3,2016,2016-08-13,2411,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,...,31.9,48.0,1.0,1.0,0.73,1.11,0.88,1.81,1.05,1.05
4,2016,2016-08-13,2411,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,...,43.6,34.6,0.0,1.0,1.11,0.68,0.84,1.60,0.00,1.05


In [37]:
#check the last five rows
matches_df.tail()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
13675,2021,2022-05-22,1869,Spanish Primera Division,Alavés,Cadiz,63.31,56.21,0.4948,0.2271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13676,2021,2022-05-22,1869,Spanish Primera Division,Granada,Espanyol,61.00,63.15,0.4044,0.3106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13677,2021,2022-05-22,1854,Italy Serie A,Sassuolo,AC Milan,64.48,75.16,0.3030,0.4546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13678,2021,2022-05-22,1854,Italy Serie A,Lazio,Verona,68.76,64.21,0.4661,0.2863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13679,2021,2022-05-22,1854,Italy Serie A,Internazionale,Sampdoria,83.64,57.58,0.7256,0.1069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
#check the info of the matches dataframe
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13680 entries, 0 to 13679
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   season       13680 non-null  int64         
 1   date         13680 non-null  datetime64[ns]
 2   league_id    13680 non-null  int64         
 3   league       13680 non-null  object        
 4   team1        13680 non-null  object        
 5   team2        13680 non-null  object        
 6   spi1         13680 non-null  float64       
 7   spi2         13680 non-null  float64       
 8   prob1        13680 non-null  float64       
 9   prob2        13680 non-null  float64       
 10  probtie      13680 non-null  float64       
 11  proj_score1  13680 non-null  float64       
 12  proj_score2  13680 non-null  float64       
 13  importance1  11994 non-null  float64       
 14  importance2  11994 non-null  float64       
 15  score1       12320 non-null  float64       
 16  scor

In [39]:
matches_df.shape

(13680, 23)

In [40]:
matches_df.columns

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object')

In [41]:
#reset the index
matches_df.reset_index(drop=True)

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-08-12,1843,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,...,32.4,67.7,0.0,1.0,0.97,0.63,0.43,0.45,0.00,1.05
1,2016,2016-08-12,1843,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,...,53.7,22.9,2.0,2.0,2.45,0.77,1.75,0.42,2.10,2.10
2,2016,2016-08-13,2411,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,...,38.1,22.2,2.0,1.0,0.85,2.77,0.17,1.25,2.10,1.05
3,2016,2016-08-13,2411,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,...,31.9,48.0,1.0,1.0,0.73,1.11,0.88,1.81,1.05,1.05
4,2016,2016-08-13,2411,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,...,43.6,34.6,0.0,1.0,1.11,0.68,0.84,1.60,0.00,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13675,2021,2022-05-22,1869,Spanish Primera Division,Alavés,Cadiz,63.31,56.21,0.4948,0.2271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13676,2021,2022-05-22,1869,Spanish Primera Division,Granada,Espanyol,61.00,63.15,0.4044,0.3106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13677,2021,2022-05-22,1854,Italy Serie A,Sassuolo,AC Milan,64.48,75.16,0.3030,0.4546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13678,2021,2022-05-22,1854,Italy Serie A,Lazio,Verona,68.76,64.21,0.4661,0.2863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
#save cleaned matches dataframe 
matches_df.to_csv('matches_df.csv')
spi_data_df.to_csv('spi_data_df.csv')

### Now I will go ahead and clean the other dataset from Kaggle

In [43]:
#read the dataset into a dataframe
soccer_df = pd.read_csv("../Data/soccer-spi/football_data_new.csv")
soccer_df.head()

,Season,date,league,team1,team2,score1,score2,results,B365_Home,B365_Draw,B365A_way,Avg<2.5,Avg>2.5
0,2016/2017,9/25/2016 10:00,Barclays Premier League,Orenburg,Ural,0,1,A,NaN,NaN,NaN,NaN,NaN
1,2016/2017,9/25/2016 12:30,Barclays Premier League,Arsenal Tula,Akhmat Grozny,0,0,D,NaN,NaN,NaN,NaN,NaN
2,2016,9/25/2016 15:00,Italy Serie A,Figueirense,Santa Cruz,3,1,H,NaN,NaN,NaN,NaN,NaN
3,2016,9/25/2016 15:00,Italy Serie A,Atletico-PR,Ponte Preta,3,0,H,NaN,NaN,NaN,NaN,NaN
4,2016/2017,9/25/2016 15:00,Barclays Premier League,Spartak Moscow,Ufa,0,1,A,NaN,NaN,NaN,NaN,NaN


In [44]:
#convert the date to datetime
soccer_df['date']=pd.to_datetime(soccer_df['date'])
soccer_df.head()

,Season,date,league,team1,team2,score1,score2,results,B365_Home,B365_Draw,B365A_way,Avg<2.5,Avg>2.5
0,2016/2017,2016-09-25 10:00:00,Barclays Premier League,Orenburg,Ural,0,1,A,NaN,NaN,NaN,NaN,NaN
1,2016/2017,2016-09-25 12:30:00,Barclays Premier League,Arsenal Tula,Akhmat Grozny,0,0,D,NaN,NaN,NaN,NaN,NaN
2,2016,2016-09-25 15:00:00,Italy Serie A,Figueirense,Santa Cruz,3,1,H,NaN,NaN,NaN,NaN,NaN
3,2016,2016-09-25 15:00:00,Italy Serie A,Atletico-PR,Ponte Preta,3,0,H,NaN,NaN,NaN,NaN,NaN
4,2016/2017,2016-09-25 15:00:00,Barclays Premier League,Spartak Moscow,Ufa,0,1,A,NaN,NaN,NaN,NaN,NaN


In [45]:
#check the shape
soccer_df.shape

(12674, 13)

In [46]:
#check the data info
soccer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12674 entries, 0 to 12673
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Season     12674 non-null  object        
 1   date       12674 non-null  datetime64[ns]
 2   league     12674 non-null  object        
 3   team1      12674 non-null  object        
 4   team2      12674 non-null  object        
 5   score1     12674 non-null  int64         
 6   score2     12674 non-null  int64         
 7   results    12673 non-null  object        
 8   B365_Home  9660 non-null   float64       
 9   B365_Draw  9660 non-null   float64       
 10  B365A_way  9660 non-null   float64       
 11  Avg<2.5    3367 non-null   float64       
 12  Avg>2.5    3367 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(5)
memory usage: 1.3+ MB


In [47]:
#split the Season column into a single date using different column 
soccer_df[['season']]=soccer_df['Season'].map(lambda x: x[:4])
soccer_df.head()

,Season,date,league,team1,team2,score1,score2,results,B365_Home,B365_Draw,B365A_way,Avg<2.5,Avg>2.5,season
0,2016/2017,2016-09-25 10:00:00,Barclays Premier League,Orenburg,Ural,0,1,A,NaN,NaN,NaN,NaN,NaN,2016
1,2016/2017,2016-09-25 12:30:00,Barclays Premier League,Arsenal Tula,Akhmat Grozny,0,0,D,NaN,NaN,NaN,NaN,NaN,2016
2,2016,2016-09-25 15:00:00,Italy Serie A,Figueirense,Santa Cruz,3,1,H,NaN,NaN,NaN,NaN,NaN,2016
3,2016,2016-09-25 15:00:00,Italy Serie A,Atletico-PR,Ponte Preta,3,0,H,NaN,NaN,NaN,NaN,NaN,2016
4,2016/2017,2016-09-25 15:00:00,Barclays Premier League,Spartak Moscow,Ufa,0,1,A,NaN,NaN,NaN,NaN,NaN,2016


In [48]:
#drop column Season since we have another one with single date
soccer_df = soccer_df.drop('Season', axis=1)
soccer_df.head()

,date,league,team1,team2,score1,score2,results,B365_Home,B365_Draw,B365A_way,Avg<2.5,Avg>2.5,season
0,2016-09-25 10:00:00,Barclays Premier League,Orenburg,Ural,0,1,A,NaN,NaN,NaN,NaN,NaN,2016
1,2016-09-25 12:30:00,Barclays Premier League,Arsenal Tula,Akhmat Grozny,0,0,D,NaN,NaN,NaN,NaN,NaN,2016
2,2016-09-25 15:00:00,Italy Serie A,Figueirense,Santa Cruz,3,1,H,NaN,NaN,NaN,NaN,NaN,2016
3,2016-09-25 15:00:00,Italy Serie A,Atletico-PR,Ponte Preta,3,0,H,NaN,NaN,NaN,NaN,NaN,2016
4,2016-09-25 15:00:00,Barclays Premier League,Spartak Moscow,Ufa,0,1,A,NaN,NaN,NaN,NaN,NaN,2016


In [49]:
#convert the season column into integer
soccer_df['season']=soccer_df['season'].astype(int)
soccer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12674 entries, 0 to 12673
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       12674 non-null  datetime64[ns]
 1   league     12674 non-null  object        
 2   team1      12674 non-null  object        
 3   team2      12674 non-null  object        
 4   score1     12674 non-null  int64         
 5   score2     12674 non-null  int64         
 6   results    12673 non-null  object        
 7   B365_Home  9660 non-null   float64       
 8   B365_Draw  9660 non-null   float64       
 9   B365A_way  9660 non-null   float64       
 10  Avg<2.5    3367 non-null   float64       
 11  Avg>2.5    3367 non-null   float64       
 12  season     12674 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(3), object(4)
memory usage: 1.3+ MB


In [50]:
#count the league
soccer_df.league.value_counts()

Barclays Premier League    4045
Italy Serie A              3490
UEFA Champions League      1929
French Ligue 1             1518
German Bundesliga          1485
Major League Soccer         207
Name: league, dtype: int64

In [51]:
#merge the matches dataframe with the soccer df
soccer_matches_df = pd.merge(matches_df,soccer_df,how ='outer')
soccer_matches_df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,nsxg1,nsxg2,adj_score1,adj_score2,results,B365_Home,B365_Draw,B365A_way,Avg<2.5,Avg>2.5
0,2016,2016-08-12,1843.0,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,...,0.43,0.45,0.00,1.05,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-08-12,1843.0,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,...,1.75,0.42,2.10,2.10,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-08-13,2411.0,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,...,0.17,1.25,2.10,1.05,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-08-13,2411.0,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,...,0.88,1.81,1.05,1.05,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-08-13,2411.0,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,...,0.84,1.60,0.00,1.05,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
soccer_matches_df.tail()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,nsxg1,nsxg2,adj_score1,adj_score2,results,B365_Home,B365_Draw,B365A_way,Avg<2.5,Avg>2.5
24745,2021,2021-05-30 20:00:00,NaN,Italy Serie A,Flamengo RJ,Palmeiras,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN
24746,2021,2021-05-30 22:15:00,NaN,Italy Serie A,Athletico-PR,America MG,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN
24747,2021,2021-05-30 22:15:00,NaN,Italy Serie A,Chapecoense-SC,Bragantino,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN
24748,2021,2021-05-30 22:15:00,NaN,Italy Serie A,Corinthians,Atletico GO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN
24749,2021,2021-05-31 00:30:00,NaN,Italy Serie A,Internacional,Sport Recife,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN


In [53]:
soccer_matches_df.shape

(24750, 29)

In [54]:
soccer_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24750 entries, 0 to 24749
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   season       24750 non-null  int64         
 1   date         24750 non-null  datetime64[ns]
 2   league_id    13680 non-null  float64       
 3   league       24750 non-null  object        
 4   team1        24750 non-null  object        
 5   team2        24750 non-null  object        
 6   spi1         13680 non-null  float64       
 7   spi2         13680 non-null  float64       
 8   prob1        13680 non-null  float64       
 9   prob2        13680 non-null  float64       
 10  probtie      13680 non-null  float64       
 11  proj_score1  13680 non-null  float64       
 12  proj_score2  13680 non-null  float64       
 13  importance1  11994 non-null  float64       
 14  importance2  11994 non-null  float64       
 15  score1       23390 non-null  float64       
 16  scor

In [55]:
soccer_matches_df.isna().sum()

season             0
date               0
league_id      11070
league             0
team1              0
team2              0
spi1           11070
spi2           11070
prob1          11070
prob2          11070
probtie        11070
proj_score1    11070
proj_score2    11070
importance1    12756
importance2    12756
score1          1360
score2          1360
xg1            12436
xg2            12436
nsxg1          12436
nsxg2          12436
adj_score1     12436
adj_score2     12436
results        12077
B365_Home      15090
B365_Draw      15090
B365A_way      15090
Avg<2.5        21383
Avg>2.5        21383
dtype: int64

In [56]:
soccer_matches_df=soccer_matches_df.drop(['Avg<2.5','Avg>2.5'],axis =1)

In [57]:
soccer_matches_df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2,results,B365_Home,B365_Draw,B365A_way
0,2016,2016-08-12,1843.0,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,...,0.97,0.63,0.43,0.45,0.00,1.05,NaN,NaN,NaN,NaN
1,2016,2016-08-12,1843.0,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,...,2.45,0.77,1.75,0.42,2.10,2.10,NaN,NaN,NaN,NaN
2,2016,2016-08-13,2411.0,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,...,0.85,2.77,0.17,1.25,2.10,1.05,NaN,NaN,NaN,NaN
3,2016,2016-08-13,2411.0,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,...,0.73,1.11,0.88,1.81,1.05,1.05,NaN,NaN,NaN,NaN
4,2016,2016-08-13,2411.0,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,...,1.11,0.68,0.84,1.60,0.00,1.05,NaN,NaN,NaN,NaN


In [58]:
soccer_matches_df.shape

(24750, 27)

In [59]:
soccer_matches_df.to_csv('soccer_matches_df.csv')

Now that the dataset is clean we can go ahead with the EDA, where we will visualize our data and drop redundant columns.